In [1]:

from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName('data_exploration') \
    .master('local[3]') \
    .enableHiveSupport() \
    .getOrCreate()

log4j:ERROR Could not read configuration file from URL [file:log4j.properties].
java.io.FileNotFoundException: log4j.properties (No such file or directory)
	at java.io.FileInputStream.open0(Native Method)
	at java.io.FileInputStream.open(FileInputStream.java:195)
	at java.io.FileInputStream.<init>(FileInputStream.java:138)
	at java.io.FileInputStream.<init>(FileInputStream.java:93)
	at sun.net.www.protocol.file.FileURLConnection.connect(FileURLConnection.java:90)
	at sun.net.www.protocol.file.FileURLConnection.getInputStream(FileURLConnection.java:188)
	at org.apache.log4j.PropertyConfigurator.doConfigure(PropertyConfigurator.java:557)
	at org.apache.log4j.helpers.OptionConverter.selectAndConfigure(OptionConverter.java:526)
	at org.apache.log4j.LogManager.<clinit>(LogManager.java:127)
	at org.slf4j.impl.Log4jLoggerFactory.<init>(Log4jLoggerFactory.java:66)
	at org.slf4j.impl.StaticLoggerBinder.<init>(StaticLoggerBinder.java:72)
	at org.slf4j.impl.StaticLoggerBinder.<clinit>(StaticLogge

In [3]:
"""creation of a dataframe from the CSV file. 
Since there are too many columns we are going to inferSchema for now and fix any columns which donot have the proper schema"""
gt_df = spark.read \
        .format('csv') \
        .option('header', 'True') \
        .option('inferSchema', 'True') \
        .load('data/globalterrorism1970-2017.csv')

In [11]:
#partition the source csv because the source csv is bigger than github allows.
gt_df.write \
    .format('csv') \
    .mode('overwrite') \
    .option('path', 'data/globalTerrorismParted') \
    .partitionBy('country_txt') \
    .save()
    

In [9]:
"""I am going to parition this dataframe now. 
Since i have multithreaded my spark session its better to create partitions so each thread can simultaneously process data.
I am going to use bucketing to achieve this"""
#just like any database we first have to create a database and select it for us to use it 
spark.sql('CREATE DATABASE IF NOT EXISTS globalTerrorism')
spark.catalog.setCurrentDatabase('globalTerrorism')
partitioned_gt_table = gt_df.write \
    .format('parquet') \
    .mode('overwrite') \
    .bucketBy(1, 'country_txt', 'iyear') \
    .saveAsTable('globalTerrorismTbl')


22/07/05 20:17:26 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
22/07/05 20:17:26 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
22/07/05 20:17:29 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
22/07/05 20:17:29 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore avinash@127.0.1.1
22/07/05 20:17:29 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException
22/07/05 20:17:29 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
22/07/05 20:17:29 WARN ObjectStore: Failed to get database globalterrorism, returning NoSuchObjectException
22/07/05 20:17:30 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringField

In [14]:
spark.sql('select attacktype1_txt, count(attacktype1_txt) from globalTerrorismTbl GROUP BY attacktype1_txt').show()

#this table is supposed to be all strings and some numbers have crept in. Also there are some unwanted strings. Lets Try to process this properly

+--------------------+----------------------+
|     attacktype1_txt|count(attacktype1_txt)|
+--------------------+----------------------+
|                   7|                    52|
|       Armed Assault|                 42566|
| the Red Hand Def...|                     1|
|     Unarmed Assault|                  1003|
|          the choice|                     1|
|                   3|                    56|
|                   8|                     5|
|                   0|                   167|
|                null|                     0|
|         and as such|                     1|
| he did manage to...|                     1|
|                   5|                     2|
|Hostage Taking (B...|                   989|
|           Hijacking|                   657|
|Hostage Taking (K...|                 11115|
|                   6|                    16|
|             Unknown|                  7267|
| have been shifte...|                     1|
|      Anwar Abu Zaid|            